<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myNifty500_Prospects_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-09-18 17:55:40


In [2]:
!pip install yfinance

In [4]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

my500 = pd.read_csv('/content/drive/My Drive/data/stocks/myNifty500Scrips.csv')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

myttm = pd.read_csv('/content/drive/My Drive/data/stocks/myNifty500-ATH-TTM.csv')

my500ss_scrips = my500[my500['StarStock']==1]['Stock'].values

mypf_scrips = mypf['Stock'].values

myttm_scrips = myttm['Stock'].values

mypf_ss = list(set(my500ss_scrips) - set(mypf_scrips))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
myss = myttm[myttm['Stock'].isin(mypf_ss)]

myss[myss['Type'] == 'Profit']

,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Latest,Stock,Max,Diff%
3,Profit,662.0,585.0,694.0,719.0,730.0,781.0,1362.0,1487.0,AIAENG.NS,1487.0,0.00
5,Profit,216.0,244.0,227.0,296.0,546.0,832.0,863.0,961.0,APLAPOLLO.NS,961.0,0.00
15,Profit,532.0,1472.0,840.0,1107.0,1620.0,1700.0,1712.0,1853.0,ADANIENSOL.NS,1853.0,0.00
25,Profit,1066.0,1096.0,1381.0,1280.0,1546.0,2287.0,5635.0,5933.0,ABCAPITAL.NS,5933.0,0.00
27,Profit,169.0,225.0,302.0,208.0,336.0,472.0,645.0,684.0,AEGISCHEM.NS,684.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
783,Profit,359.0,-6607.0,-3933.0,-4151.0,2327.0,8566.0,12147.0,15162.0,UNIONBANK.NS,15162.0,0.00
813,Profit,-12.0,13.0,35.0,-9.0,-129.0,-2.0,149.0,158.0,WESTLIFE.NS,158.0,0.00
821,Profit,300.0,382.0,411.0,223.0,152.0,196.0,427.0,474.0,ZFCVINDIA.NS,474.0,0.00
825,Profit,349.0,352.0,445.0,376.0,433.0,574.0,444.0,552.0,ZENSARTECH.NS,574.0,3.83


In [18]:
import yfinance as yfin
#import talib
yfin.pdr_override()
#import pandas_ta as ta
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      #stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      #stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in mypf_ss:
      print(stock_id)
      stock_id = stock_id.upper()
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      stock_id = stock_id.removesuffix('.NS')
      #current_pe, current_roce, current_roe = get_current_features(stock_id)
      #tmp['Curr_PE'] = current_pe
      #tmp['Curr_ROCE%'] = current_roce
      #tmp['Curr_ROE%'] = current_roe
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

ABCAPITAL.NS


<ipython-input-18-45f998439741>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prec_dev = df_prec_dev.append(tmp)


ADANIENSOL.NS


<ipython-input-18-45f998439741>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prec_dev = df_prec_dev.append(tmp)


AEGISCHEM.NS


...

<ipython-input-18-45f998439741>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prec_dev = df_prec_dev.append(tmp)


ZENSARTECH.NS


<ipython-input-18-45f998439741>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prec_dev = df_prec_dev.append(tmp)


ZFCVINDIA.NS


<ipython-input-18-45f998439741>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prec_dev = df_prec_dev.append(tmp)


In [24]:
#df_common_features.drop(['Date'], axis=1, inplace=True)
df_common_features['Upside%Target'] = round((df_common_features['Max']-df_common_features['Close'])*100/(df_common_features['Close']),0)
df_common_features.sort_values(by = 'Upside%Target', ascending=False).head(25)

,Close,200_SMA,Dev%_200,Stock,Max,Upside%Target
1234,18.0,23.0,-20.22,BCG.NS,194.0,978.0
1234,230.0,175.0,31.57,RBLBANK.NS,688.0,199.0
613,833.0,620.0,34.39,NAZARA.NS,1601.0,92.0
1234,177.0,205.0,-13.66,DELTACORP.NS,330.0,86.0
1234,133.0,141.0,-5.64,CUB.NS,239.0,80.0
1234,1246.0,612.0,103.65,BSE.NS,2176.0,75.0
1233,1492.0,1582.0,-5.68,TCIEXP.NS,2501.0,68.0
1043,3026.0,2644.0,14.46,INDIAMART.NS,4921.0,63.0
1234,708.0,677.0,4.54,JUSTDIAL.NS,1111.0,57.0
1234,1621.0,1361.0,19.13,GODREJPROP.NS,2503.0,54.0
